In [1]:
import pandas as pd
import json
import ast

In [2]:
pd.set_option('display.max_colwidth', None) ##para ver todo el contenido de la columna

In [3]:
# traigo el dataset credits a un dataframe y le cambio el nombre de la columna del id(pelicula) a id_original
dfcredits= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\credits\credits.csv')
dfcredits.rename(columns={'id':'id_original'}, inplace= True)


In [4]:
#utilizo ast literal para evaluar y transformar las cadenas de texto que representan diccionarios a diccionarios propiamente dichos
dfcredits['crew'] = dfcredits['crew'].apply(ast.literal_eval) 

In [5]:
df_exploded = dfcredits.explode('crew')

def extract_crew_info(x):
    if isinstance(x, dict):
        return pd.Series([x.get('credit_id', ''), x.get('department', ''),x.get('gender', ''),x.get('id', ''),x.get('job', ''),x.get('name', ''),x.get('profile_path', '')  ])
    else:
        return pd.Series(['', ''])

df_exploded[['credit_id', 'department', 'gender','id_crew','job', 'name', 'profile_path']] = df_exploded['crew'].apply(extract_crew_info) 

In [6]:
df = dfcredits.join(df_exploded[['credit_id', 'department', 'gender', 'id_crew', 'job', 'name', 'profile_path']]) 

In [7]:
columna_eliminada = ['crew','cast', 'credit_id','profile_path', 'gender', 'id_crew', 'department']
df.drop(columns=columna_eliminada, inplace=True) 




In [8]:
df.head()

,id_original,job,name
0,862,Director,John Lasseter
0,862,Screenplay,Joss Whedon
0,862,Screenplay,Andrew Stanton
0,862,Screenplay,Joel Cohen
0,862,Screenplay,Alec Sokolow


In [9]:
# Convertir la columna numéricas a cadenas

df['id_original'] = df['id_original'].astype(str) 

In [10]:
director_resultado = df[df['job'] == 'Director']['job'].value_counts()
director_resultado 

job
Director    49048
Name: count, dtype: int64

In [11]:
df_director= df[df['job']== 'Director']
df_director.head()

,id_original,job,name
0,862,Director,John Lasseter
1,8844,Director,Joe Johnston
2,15602,Director,Howard Deutch
3,31357,Director,Forest Whitaker
4,11862,Director,Charles Shyer


In [12]:
df_director.shape

(49048, 3)

In [31]:
grouped_dfdirector = df_director.groupby('id_original').agg({'name': lambda x: "" + ",".join(x) + "" }).reset_index(False)

grouped_dfdirector

,id_original,name
0,100,Guy Ritchie
1,10000,Sergio Cabrera
2,10001,Yahoo Serious
3,100010,Frank Borzage
4,100017,Angelina Maccarone
...,...,...
44540,99946,Sam Taylor
44541,9995,Robert Adetuyi
44542,9997,Shane Abbess
44543,99977,Dom DeLuise


In [33]:
grouped_dfdirector.to_csv('directoresAgrupadosID.csv', index=False)

In [34]:
# identificar Director por cada Id Pelicula

id_director = df_director[df_director['id_original'] == '862']
director_names = id_director['name'].values[0].split(',')
director_name = director_names[0]

print(f"ID: 862, Director: {director_name}")


ID: 862, Director: John Lasseter


In [15]:
## Para armar la funcion de director creo aca un nuevo DATASET
# # Me traigo el de movies small y lo uno con el que creo aca
dfmovies= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\A-ETLconsignas\MoviesSMALL.csv')

In [16]:
# Me quedo con las columnas que voy a usar 
eliminar= ['original_language', 'overview', 'popularity', 'runtime', 'release_year' ]
dfmovies1 = dfmovies.drop(columns=eliminar)
dfmovies1.head()

,budget,id,release_date,revenue,title,return
0,30000000.0,862,1995-10-30,373554033.0,Toy Story,12.451801
1,65000000.0,8844,1995-12-15,262797249.0,Jumanji,4.043035
2,0.0,15602,1995-12-22,0.0,Grumpier Old Men,NaN
3,16000000.0,31357,1995-12-22,81452156.0,Waiting to Exhale,5.090760
4,0.0,11862,1995-02-10,76578911.0,Father of the Bride Part II,inf


In [17]:
dfmovies1.shape

(44936, 6)

In [18]:
# Lo transformo en tipo de dato str para que sea el mismo tipo de dato de la columna que quiero unir
# y le pongo el mismo nombre
dfmovies1['id_original'] = dfmovies1['id'].astype(str) 
dfmovies1.drop(columns='id', inplace=True)

In [19]:
#uno los dos datasets
mergedf= pd.merge(df_director, dfmovies1, on='id_original', how='left')
mergedf.head()

,id_original,job,name,budget,release_date,revenue,title,return
0,862,Director,John Lasseter,30000000.0,1995-10-30,373554033.0,Toy Story,12.451801
1,8844,Director,Joe Johnston,65000000.0,1995-12-15,262797249.0,Jumanji,4.043035
2,15602,Director,Howard Deutch,0.0,1995-12-22,0.0,Grumpier Old Men,NaN
3,31357,Director,Forest Whitaker,16000000.0,1995-12-22,81452156.0,Waiting to Exhale,5.090760
4,11862,Director,Charles Shyer,0.0,1995-02-10,76578911.0,Father of the Bride Part II,inf


In [20]:
mergedf.shape

(49118, 8)

In [21]:
#agrupo y selecciono dos columnas para saber el return por director. y creo otro data frame
grouped= mergedf[['name','return']].groupby('name').sum().reset_index()
#le cambio el nombre a la columna para que no sea igual cuando lo una al otro
grouped.rename(columns={'return': 'return_dire'}, inplace=True)

In [22]:
grouped.head()

,name,return_dire
0,Dale Trevillion\t,0.00000
1,Davide Manuli,0.00000
2,E.W. Swackhamer,0.00000
3,Vitaliy Vorobyov,0.00000
4,Yeon Sang-Ho,0.24147


In [23]:
grouped.shape

(19740, 2)

In [24]:
## uno los dos data frames que acabo de crear para luego usarlos en la funcion de DIRECTOR
mergedbis= pd.merge(mergedf, grouped, on='name', how='left')
mergedbis.head()

,id_original,job,name,budget,release_date,revenue,title,return,return_dire
0,862,Director,John Lasseter,30000000.0,1995-10-30,373554033.0,Toy Story,12.451801,27.654378
1,8844,Director,Joe Johnston,65000000.0,1995-12-15,262797249.0,Jumanji,4.043035,20.679024
2,15602,Director,Howard Deutch,0.0,1995-12-22,0.0,Grumpier Old Men,NaN,NaN
3,31357,Director,Forest Whitaker,16000000.0,1995-12-22,81452156.0,Waiting to Exhale,5.090760,5.090760
4,11862,Director,Charles Shyer,0.0,1995-02-10,76578911.0,Father of the Bride Part II,inf,NaN


In [25]:
#finalmente le saco las columnas id y job que no las necesito
eliminarcol= ['id_original', 'job']
mergedbis.drop(columns=eliminarcol, inplace=True)

In [26]:
mergedbis.head()

,name,budget,release_date,revenue,title,return,return_dire
0,John Lasseter,30000000.0,1995-10-30,373554033.0,Toy Story,12.451801,27.654378
1,Joe Johnston,65000000.0,1995-12-15,262797249.0,Jumanji,4.043035,20.679024
2,Howard Deutch,0.0,1995-12-22,0.0,Grumpier Old Men,NaN,NaN
3,Forest Whitaker,16000000.0,1995-12-22,81452156.0,Waiting to Exhale,5.090760,5.090760
4,Charles Shyer,0.0,1995-02-10,76578911.0,Father of the Bride Part II,inf,NaN


In [27]:
mergedbis.shape

(49118, 7)

In [28]:
#elimino nulos 
mergedbis.dropna(inplace=True)

In [29]:
mergedbis.shape

(8398, 7)

In [30]:
### lo transformo en un CSV y elimino el indice
mergedbis.to_csv('DirectorFUNCION.csv', index=False)